In [1]:
# 댓글을 달 빈 리스트를 생성합니다.

# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup 
import requests 
import re 
import sys 
import pprint 
import pandas as pd
from datetime import date, timedelta
import time
import numpy as np 

class NaverCommentsCrawler:
    
    def __init__(self,start_date=None, end_date=None):
        self.idx_result_df = pd.DataFrame()
        self.filtered_idx_result_df = pd.DataFrame()
        self.t_list = []
        
        input_file_name = '/media/Data/Naver_News/result/indexing.txt'
        
        # 인덱싱 파일 읽기
        self.idx_result_df = pd.read_csv(input_file_name,encoding="UTF8",header=None)
        self.idx_result_df.columns = ['date','title','news','content_url_list','file_list']

        self.start_date = start_date
        self.end_date = end_date

        # 시작날짜를 지정하지 않을경우 20180302
        if(self.start_date == None):
            self.start_date = '20100101'

        # 끝 날짜를 지정하지 않을경우 어제까지 수집
        if(self.end_date == None):
            yesterday = date.today() - timedelta(1)
            self.end_date = str('20191231')
            
        dt_index = pd.date_range(start=self.start_date, end = self.end_date)
        dt_list = dt_index.strftime("%Y-%m-%d").tolist()

        idx = list(map(lambda x: x in dt_list, self.idx_result_df['date'].tolist()))

        self.filtered_idx_result_df = self.idx_result_df.loc[idx]
    
    def crawling_comments(self,url):
        
        List=[] 
        oid=url.split("oid=")[1].split("&")[0] 
        aid=url.split("aid=")[1] 
        page=1     
        header = { 
          "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36", 
          "referer":url, 
        } 
        
        while True : 
            c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE"  
            # 파싱하는 단계입니다.
            r=requests.get(c_url,headers=header) 
            cont=BeautifulSoup(r.content,"html.parser")     
            total_comm=str(cont).split('comment":')[1].split(",")[0] 

            match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont)) 
            # 댓글을 리스트에 중첩합니다.
            List.append(match) 
            # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
            if int(total_comm) <= ((page) * 20): 
                break 
            else :  
                page+=1
        
        if(len(List) == 0):
            f = ''
            return f
        
        f = self.flatten(List)      
        f = " @&/&@ ".join(f)
        return f

    def crawling_all(self):
        url_list =  self.filtered_idx_result_df['content_url_list']  
        t_list = []
        sec = 10
        i = 1
        
        for u in url_list:
            if(i % 2000 == 0):
                time.sleep(sec)
                print(i)
            i= i + 1

            try:
                cc = self.crawling_comments(u)
            except:
                cc = ''
            
            t_list.append(cc)
            
        self.t_list = t_list
            
    
    # 여러 리스트들을 하나로 묶어 주는 함수입니다.
    def flatten(self,l): 
        flatList = [] 
        for elem in l: 
          # if an element of a list is a list 
          # iterate over this list and add elements to flatList  
            if type(elem) == list: 
                for e in elem: 
                    flatList.append(e) 
            else: 
                flatList.append(elem) 
        return flatList

In [2]:
ncc = NaverCommentsCrawler(start_date="2020-01-01",end_date='2020-01-03')

In [3]:
ncc.crawling_all()

In [4]:
ncc.filtered_idx_result_df["comments"] = ncc.t_list 
#ncc.filtered_idx_result_df = ncc.filtered_idx_result_df[ncc.filtered_idx_result_df["comments"] != '']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
ncc.filtered_idx_result_df

,date,title,news,content_url_list,file_list,comments
380364,2020-01-01,"伊 대통령 신년사 ""기후변화 대응, 더 늦어서는 안 돼""",연합뉴스,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200101_0.txt,
380365,2020-01-01,[날씨] 중부지방 곳곳 오전에 눈발…미세먼지 '보통'∼'나쁨',연합뉴스,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200101_1.txt,"""국외미세먼지가 아니고 중국발이다."" @&/&@ ""미세먼지가 아니라 미세중금속이다..."
380366,2020-01-01,내일도 영하권 강추위…수도권 눈발에 미세먼지,뉴시스,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200101_2.txt,"""왜 너네 기사만 강추위냐?"""
380367,2020-01-01,길냥이에게 따뜻한 보금자리를...[김하국의 애니멀세대],스포츠서울,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200101_3.txt,"""너무 가슴아프면서 가슴 따뜻해지는 기사입니다. 가끔 저희 동네에서도 길냥이들을 보..."
380368,2020-01-01,파주서 야생 멧돼지 ASF바이러스 검출…전국 56번째,연합뉴스,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200101_4.txt,
...,...,...,...,...,...,...
380829,2020-01-03,대전·세종·충남 구름 많고 온화…미세먼지 '나쁨',연합뉴스,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200103_114.txt,
380830,2020-01-03,"전국 대부분 미세먼지 '나쁨'…""추위 주춤하나 일교차 주의""",연합뉴스,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200103_115.txt,"""어제도 온통 잿빛이였다.문죄인 공약이다 미세먼지 시진핑 만나서 해결한다고했는데 이..."
380831,2020-01-03,[단독] “4대강 보 처리방안 내달 말까지 결론”,한겨레,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200103_116.txt,"""4대강보 해체? 문재인 대통령은 국정파괴범이다. 공수처로 헌법을 유린했고 탈원전으..."
380832,2020-01-03,오늘 수도권 미세먼지 예비저감조치,국민일보,https://news.naver.com/main/read.nhn?mode=LS2D...,./result/20200103_117.txt,"""국민청원 중국발 초미세먼지 청원 글 올려도 지들이 걸러 버려서 아에 올라가질 않는..."


In [8]:
ncc.filtered_idx_result_df.to_excel("./2020_2020_E.xlsx")

/usr/local/lib/python3.6/dist-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.7.3' currently installed).
  warnings.warn(msg, UserWarning)
